## Uniform Cost Search 

In [2]:
import heapq

def ucs_search(grid, start):
    corners = [(1, 1), (1, len(grid[0])-2), (len(grid)-2, 1), (len(grid)-2, len(grid[0])-2)]
    goal = min(corners, key=lambda x: abs(start[0] - x[0]) + abs(start[1] - x[1]))
    open_set = []
    heapq.heappush(open_set, (0, start))  # Only g_score
    came_from = {}
    g_score = {start: 0}

    while open_set:
        current_g, current = heapq.heappop(open_set)
        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in neighbors(grid, current):
            tentative_g_score = g_score[current] + 1  # Assuming all movements have a cost of 1
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                heapq.heappush(open_set, (g_score[neighbor], neighbor))


## Greedy Search

In [3]:
import heapq

def greedy_best_first_search(grid, start):
    corners = [(1, 1), (1, len(grid[0])-2), (len(grid)-2, 1), (len(grid)-2, len(grid[0])-2)]
    goal = min(corners, key=lambda x: abs(start[0] - x[0]) + abs(start[1] - x[1]))
    open_set = []
    heapq.heappush(open_set, (heuristic_cost_estimate(start, goal), start))  # Only h_score
    came_from = {}

    while open_set:
        current_h, current = heapq.heappop(open_set)
        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in neighbors(grid, current):
            if neighbor not in came_from:  # Check if the neighbor is not already explored
                came_from[neighbor] = current
                heapq.heappush(open_set, (heuristic_cost_estimate(neighbor, goal), neighbor))
